# Analysis of datasets - source: WorkBench model

Mission: verify the integrity of a system model

Objectives: 
- use pattern matching to detect the inconsistencies,
- use graph analysis to display correction suggestions,
- allocate the correction of the system model to a user, selected as being familiar with the CAESAR WorkBench v1.23,
- assume that the test is finished when the graph analysis returns that the input model complies with the 3 different rules.

Note: the RDF dataset shall be 
- renamed: *.trig
- available through a query endpoint

In [43]:
# import necessary libraries 
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import Javascript

from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

In [44]:
# definition of a function: sparql query
def query_fusekiData(sparql_query, sparql_service_url):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to the endpoint
    sparql = SPARQLWrapper(sparql_service_url)
    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # ask for the result
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [45]:
# definition of ipywidget 'Text' to prompt the  endpoint to query
layout = widgets.Layout(width='auto', height='40px') #set width and height of the different widgets
endpoint=widgets.Text(
    value='http://localhost:3030/WBv13.5',
    placeholder='http://localhost:3030/',
    description='Endpoint: ',
    layout = layout,
    disabled=False
)
display(endpoint)

Text(value='http://localhost:3030/WBv13.5', description='Endpoint: ', layout=Layout(height='40px', width='auto…

In [46]:
# definition of ipywidget 'Button' to take into account the endpoint value prompted
output = widgets.Output()
def on_button_clicked(b):
    with output:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        
b = widgets.Button(
    description='Analyze dataset', 
    layout = layout,
    disabled=False)
display(b)
b.on_click(on_button_clicked)

Button(description='Analyze dataset', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

In [47]:
# verification of the endpoint URL
req = Request(endpoint.value)
try:
    response = urlopen(req)
except HTTPError as e:
    print('Error - Endpoint: ' +endpoint.value + ' not found. Verify URL. (Error code: ', e.code, ')')
    raise SystemExit
except URLError as e:
    print('Error - Server not reached. (Reason: ', e.reason, ')')
    raise SystemExit
else:
    print ('Endpoint: ' +endpoint.value + ' found. Processing results now...')

Endpoint: http://localhost:3030/WBv13.5 found. Processing results now...


# Rule #1 : Each subsystem shall be supplied by exactly one work package

In [48]:
# sparql workpackage query
workpackageQuery = """
PREFIX project: <http://imce.jpl.nasa.gov/foundation/project/project#>
PREFIX oml: <http://def.seegrid.csiro.au/ontology/om/om-lite#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fse:   <http://imce.jpl.nasa.gov/discipline/fse/fse#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX analysis: <http://imce.jpl.nasa.gov/foundation/analysis/analysis#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>
PREFIX pr: <http://purl.org/ontology/prv/core#>
PREFIX pro: <http://purl.org/hpi/patchr#>

SELECT DISTINCT ?subSystem ?workPackage

FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a fse:Subsystem ;
     base:hasCanonicalName ?subSystem .
  
  optional {
    ?b a project:WorkPackage ;
       base:hasCanonicalName ?workPackage ;
       project:supplies ?a .
  } 
}
"""
if query_fusekiData(workpackageQuery, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(workpackageQuery, endpoint.value)

In [49]:
if 'workPackage.value' not in result_table.columns:
    print('Error - No WorkPackages found. Verify OWL data.')
else:
    obj = result_table[["subSystem.value", "workPackage.value"]]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

## Problem is splitted into 2 subparts:

### a/ Is there subsytems not supplied by any WorkPackage?

In [50]:
sol_a = obj.workPackage.isnull().sum() # select rows based on condition 

print('Number of subsystems not supplied by any WorkPackage: '+str(sol_a))
sol_a = obj[obj['workPackage'].isnull()] 
sol_a = sol_a['subSystem'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently not supplied by any WorkPackage: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these subsystems to one WorkPackage, in the Mass Manifest.")

Number of subsystems not supplied by any WorkPackage: 2

List of subsystems currently not supplied by any WorkPackage: 
- Communications
- Thermal control

Suggestion: Connect each of these subsystems to one WorkPackage, in the Mass Manifest.


### b/ Is there subystems supplied by more than one WorkPackage?

In [51]:
# generate "supplies" relationships between a workPackage and a subsystem
#obj = obj.append({'subSystem' : 'Payload' , 'workPackage' : 'WPx'} , ignore_index=True)
#obj = obj.append({'subSystem' : 'EPS' , 'workPackage' : 'WPx'} , ignore_index=True)

In [52]:
sol_b = obj.groupby(["subSystem"]).size()
sol_b = sol_b.to_frame("workPackage_count")

sol_b = sol_b[sol_b['workPackage_count'] > 1] # select rows based on condition 
print('Number of subsystems supplied by more than one WorkPackage: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently supplied by more than one WorkPackage: ')
    for i in range(sol_b.shape[0]):
        print('- Subsystem ' +list(sol_b.index)[i] +' has relationship(s) with ' +str(list(sol_b.workPackage_count)[i]) +' different WorkPackages: ')
        for j in range((obj[obj['subSystem'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['subSystem'] == list(sol_b.index)[i]].workPackage)[j])
    print("\nSuggestion: Look for these subsytems in the Mass Manifest and delete the extra relationships.")

Number of subsystems supplied by more than one WorkPackage: 0
The model complies with the rule. No action needs to be performed.


# Rule #2 : Each power load assembly shall be connected to at least one power source assembly

Note:
1. Solar arrays provide power to the EPS regulators and converters
2. EPS regulators and converters provide power to the batteries
3. Batteries provide power to the EPS regulators and converters 
4. EPS regulators and converters provide power to all the other power load components

In [53]:
# sparql powerAssembly query
powerAssemblyQuery = """
PREFIX oml: <http://def.seegrid.csiro.au/ontology/om/om-lite#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fse:   <http://imce.jpl.nasa.gov/discipline/fse/fse#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX analysis: <http://imce.jpl.nasa.gov/foundation/analysis/analysis#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?powerLoadAssembly ?loadAssemblyID ?functionID ?powerSourceAssembly ?sourceAssemblyID
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a fse:Assembly ;
     base:hasAnnotations "Power load" ;
     fse:hasAssemblyReferenceDesignator ?loadAssemblyID ;                                
	 base:hasCanonicalName ?powerLoadAssembly .

  optional {
  ?d a fse:Assembly ;
     base:hasAnnotations "Power source" ;
     fse:hasAssemblyReferenceDesignator ?sourceAssemblyID ;
	 base:hasCanonicalName ?powerSourceAssembly .
  
  ?c a mission:Junction ;
     fse:hasFunctionNumber ?functionID ;
     fse:connectsAssembly ?a ;
     fse:connectsAssembly ?d . 
  }     
}
"""
if query_fusekiData(powerAssemblyQuery, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(powerAssemblyQuery, endpoint.value)

Error - The pattern matching query returned no results. Verify OWL data.


SystemExit: 

In [ ]:
if 'loadAssemblyID.value' not in result_table.columns:
    print('Error - No Power Load Assemblies found. Verify OWL data.')
else:
    obj = result_table[["powerLoadAssembly.value", "loadAssemblyID.value", "functionID.value", "powerSourceAssembly.value", "sourceAssemblyID.value"]]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [ ]:
sol = obj.functionID.isnull().sum() # select rows based on condition 
print('Number of power load assemblies not connected to any power source assembly: '+str(sol))

if sol==0:
    print('The model complies with the rule. No action needs to be performed.')
else:
    sol = obj[obj['functionID'].isnull()] 
    sol = sol[["powerLoadAssembly", "loadAssemblyID"]]
    print('\nList of power load assemblies not connected to any power source assembly: ' )
    for i in range(sol.shape[0]):
        print('Name: ' +sol.iloc[i, 0] +' - ID: ' +sol.iloc[i, 1])
    print('\nSuggestion: Use functions to connect each of these assemblies to at least one power source assembly, from the Function List Table.')

# Rule #3 : Each power load end circuit interface shall be connected to one power source end circuit interface

In [ ]:
# sparql powerInterface query
powerInterfaceQuery = """
PREFIX oml: <http://def.seegrid.csiro.au/ontology/om/om-lite#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fse:   <http://imce.jpl.nasa.gov/discipline/fse/fse#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX analysis: <http://imce.jpl.nasa.gov/foundation/analysis/analysis#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?powerLoadAssembly ?powerLoadAssemblyID ?powerLoadInterface ?functionID 
FROM <urn:x-arq:UnionGraph>
WHERE
{
    ?c a fse:Assembly ;
       base:hasAnnotations "Power load" ;
       fse:hasReferenceDesignator ?powerLoadAssemblyID ;
       base:hasCanonicalName ?powerLoadAssembly .
      
   
  optional 
  {
    ?a a <http://io.openceasar.firesat/efse/libraries/ECTypes#ID_oj74HBJ8czeoHVJe6MXGCvU94TQaXQzNM6wDjYko67NKxLaD5VQCW7zRrs2p5CxcF75J> ; 
    base:hasCanonicalName ?powerLoadInterface .
    
    ?b a fse:Function ;
    fse:hasFunctionNumber ?functionID ;
    mission:joins ?a .
    
    ?c mission:presents ?a .
   }
}
"""
if query_fusekiData(powerInterfaceQuery, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(powerInterfaceQuery, endpoint.value)    

In [ ]:
if 'powerLoadAssembly.value' not in result_table.columns:
    print('Error - No Power Load Assemblies found. Verify OWL data.')
else:
    obj = result_table[["functionID.value", "powerLoadAssembly.value", "powerLoadAssemblyID.value", "powerLoadInterface.value"]]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [ ]:
sol = obj.powerLoadInterface.isnull().sum() # select rows based on condition 
print('Number of power load end circuit interfaces not connected to any power source end circuit interface: '+str(sol))

if sol==0:
    print('The model complies with the rule. No action needs to be performed.')
else:
    sol = obj[obj['powerLoadInterface'].isnull()] 
    sol = sol[["powerLoadAssembly", "powerLoadAssemblyID"]]
    print('\nList of the power load assemblies with a missing interface connection with a power source assembly: ')
    for i in range(sol.shape[0]):
        print('Name: ' +sol.iloc[i, 0] +' - ID: ' +sol.iloc[i, 1])
    print('\nSuggestion: Look for these assemblies and connect their end circuit interfaces to one interface of at least a power source assembly, from the Function List Table.')